In [11]:
%pip install pywt
import os
import sys

import pandas as pd
import numpy as np

import pywt
import scipy.io as spio
from scipy.stats import entropy
from collections import Counter

from sklearn import svm
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

import timeit

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pywt (from versions: none)
ERROR: No matching distribution found for pywt


In [12]:
'''
   Authors: Shivam Chaudhary
            Centre of Behavioural and Cognitive Sciences, Allahabad
   In this project we will be recognising Emotion of a Human being from EEG signal.
   About the data set : The data set is called the seed data set.
   It contains data of 15 people that underwent trails 15 times each thrice.

           Total data items =  15 (subjects) * 15 (trials each) * 3 (sessions each)
                            = 675 data items

   Our project consists of 4 modules, namely : pre processing, feature extraction, feature reduction and classification,
   all of which are mentioned in detail in the black book.

'''

'\n   Authors: Shivam Chaudhary\n            Centre of Behavioural and Cognitive Sciences, Allahabad\n   In this project we will be recognising Emotion of a Human being from EEG signal.\n   About the data set : The data set is called the seed data set.\n   It contains data of 15 people that underwent trails 15 times each thrice.\n\n           Total data items =  15 (subjects) * 15 (trials each) * 3 (sessions each)\n                            = 675 data items\n\n   Our project consists of 4 modules, namely : pre processing, feature extraction, feature reduction and classification,\n   all of which are mentioned in detail in the black book.\n\n'

In [13]:
WAVELET = "db6"
MAX_LEVEL = 5

In [ ]:
dir = "/content/drive/MyDrive/EEG-emotion-python/seed dataset/SEED"
os.chdir(dir)
os.getcwd()

: 

In [ ]:
# Unzip the data file
# # Do only once
# import zipfile
# with zipfile.ZipFile("ExtractedFeatures.zip", 'r') as zip_ref:
#     zip_ref.extractall(os.getcwd() + "/temp")
# !unzip -u "/content/drive/MyDrive/EEG-emotion-python/seed dataset/SEED/Preprocessed_EEG.zip" -d "/content/drive/MyDrive/EEG-emotion-python/seed dataset/SEED/temp"

: 

In [ ]:
# reading the channel order for dataframe
channel_order = pd.read_excel("Channel Order.xlsx", header=None)
channel_order.head()

: 

In [ ]:
# extract labels file
labels = spio.loadmat("label.mat")
labels_df = pd.DataFrame(np.hstack(labels["label"]))
labels_df.head()

: 

In [ ]:
files = os.listdir("temp/Preprocessed_EEG/")
files

: 

**Feature extraction**

In [ ]:
def calc_wavelet_energy(data_set):
  """
    Input : 1 * N vector
    Output: Float with the wavelet energy of the input vector,
    rounded to 3 decimal places.
  """
  # p_sqr = [i ** 2 for i in data_set]
  wavelet_energy = np.nansum(np.log2(np.square(data_set)))
  return round(wavelet_energy, 3)

: 

In [ ]:
    def calc_shannon_entropy(data_set):
        """
            Input : 1 * N vector
            Output: Float with the wavelet entropy of the input vector,
                    rounded to 3 decimal places.
        """
        # probability = [i ** 2 for i in data_set]
        probability = np.square(data_set)
        shannon_entropy = -np.nansum(probability * np.log2(probability))
        return round(shannon_entropy, 3)

: 

In [ ]:
participant_trial = []
features_table = pd.DataFrame(columns=range(620))
for file in files:
  mat_file = spio.loadmat("temp/Preprocessed_EEG/" + file)
  keys = [key for key, values in mat_file.items() if key != '__header__' and key != '__version__' and key != '__globals__' ]
  for data_file in keys:
    data_df = pd.DataFrame(mat_file[data_file])
    # print(data_df)
    M = data_df.shape[0]
    N = data_df.shape[1]
    # Feature extraction part of the module
    Entropy = []
    Energy = []
    for channel in data_df.iterrows():  # Iterating through the 62 channels
      dwt_bands = []
      data = channel[1]
      int_ent = []
      int_eng = []
      for band in range(MAX_LEVEL):
        (data, coeff_d) = pywt.dwt(data, WAVELET)
        dwt_bands.append(coeff_d)
      
      for band in range(len(dwt_bands)):  # DWT_bands = 23504, 11755
        int_ent.append(calc_shannon_entropy(dwt_bands[len(dwt_bands) - band - 1]))
        int_eng.append(calc_wavelet_energy(dwt_bands[len(dwt_bands) - band - 1]))
      
      Entropy.append(int_ent)
      Energy.append(int_eng)
    
    unroll_entropy = []
    unroll_energy = []
    '''
        Transforming 2D array into 1D vector of features and then 
        combining the two 1D arrays.
    '''
    for i in range(len(Entropy)):
      for j in range(len(Entropy[0])):
        unroll_entropy.append(Entropy[i][j])
        unroll_energy.append(Energy[i][j])

    features = unroll_entropy + unroll_energy
    participant_trial.append(features)
    features_table.loc[len(features_table.index)] = features
    # print(data_file)
    # print(features)
  print(file)


: 

In [ ]:
file_dir = dir + "/temp/analysis/"

: 

In [ ]:
features_table.to_csv(file_dir + "features" + WAVELET + ".csv", index=False)

: 

**Principal Components Analysis**

In [ ]:
data = pd.read_csv(file_dir + "features" + WAVELET + ".csv")

: 

In [ ]:
# 1. Normalising data and getting transpose
normalised = pd.DataFrame(normalize(data, axis = 0))

# 2. Finding covariance matrix
covariance_df = normalised.cov()

# 3. Eigen Vectors
u, s, v = np.linalg.svd(covariance_df)

# 4. Principal Components
data_reduced = normalised @ u
data_reduced.head()

: 

In [ ]:
data_reduced.to_csv(file_dir + "pc" + WAVELET + ".csv", index=False)

: 

In [ ]:
file_dir

: 

**Running data through classifiers**


1.   SVM



In [ ]:
# Reading data and splitting 
pcs = pd.read_csv(file_dir + "pc" + WAVELET + ".csv")
outputs = pd.read_csv(file_dir + "outputs_main.csv", header=None)

X = pcs.iloc[:, :].values
Y = outputs.iloc[:, :].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)

: 

In [ ]:
svc = SVC()
parameters = {"C": (100, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9), "gamma": (1e-08, 1e-7, 1e-6, 1e-5)}
grid_search = GridSearchCV(svc, parameters, n_jobs=-1, cv=5)
start_time = timeit.default_timer()
grid_search.fit(X_train, Y_train)
print("--- {0:.3f} seconds ---".format(timeit.default_timer() - start_time))
print(grid_search.best_params_)
svc_best = grid_search.best_estimator_
accuracy = svc_best.score(X_test, Y_test)
print("Accuracy on the testing set is: {0:.1f}%".format(accuracy*100))
prediction = svc_best.predict(X_test)
report = classification_report(Y_test, prediction)
print(report)


: 

: 